# Jupyter Notebook: GNSS VOD Toy Anomaly Dataset
I'll convert the script into a well-structured Jupyter notebook with detailed explanations of each step in the GNSS VOD analysis process

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from analysis.calculate_anomaly_functions import ke_fun
from analysis.calculate_anomalies import calculate_anomaly
import gnssvod as gv
from processing.export_vod_funs import plot_hemi

ImportError: cannot import name 'ke_fun' from 'analysis.calculate_anomalies' (/home/konsch/Documents/5-Repos/gnssvod/gnssvod/analysis/calculate_anomalies.py)

## 1. Define Key Functions
First, let's define the extinction coefficient function that calculates the effective extinction coefficient (ke) and pathlength based on VOD, canopy height, and elevation angle.

In [ ]:
"""
theta = 90 - elevation  # convert elevation to zenith angle
pathlength = d / np.cos(np.deg2rad(theta))
ke = vod / pathlength
"""

In [ ]:
# Set parameters
d = 20  # canopy height in meters

# Create a timeline of observations
dates = pd.date_range(start='2022-01-01', periods=100, freq='10min')

# Define satellite vehicles
svs = [f'SV{i}' for i in range(1, 6)]

# Generate random data
vod_dict = {
    'Epoch': np.tile(dates, len(svs) * len(dates)),
    'SV': np.repeat(svs, len(dates) * len(dates)),
    'VOD1': np.random.normal(loc=1, scale=0.3, size=len(dates) * len(svs) * len(dates)),
    'Azimuth': np.random.rand(len(dates) * len(svs) * len(dates)) * 360,
    'Elevation': np.random.rand(len(dates) * len(svs) * len(dates)) * 90
}

# Create the dataframe
vod = pd.DataFrame(vod_dict)

# Examine the data distributions
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
vod.plot(kind='hist', y='VOD1', bins=30, title='VOD1 Distribution', ax=axes[0])
vod.plot(kind='hist', y='Azimuth', bins=30, title='Azimuth Distribution', ax=axes[1])
vod.plot(kind='hist', y='Elevation', bins=30, title='Elevation Distribution', ax=axes[2])
plt.tight_layout()
plt.show()

In [ ]:
# Build a hemispheric grid with 1-degree resolution and 10-degree elevation cutoff
hemi = gv.hemibuild(1, 10)

# Classify VOD observations into grid cells
vod_cell = hemi.add_CellID(vod.copy())

# Display a sample of the classified data
print("Sample of VOD data with CellID:")
print(vod_cell.head())

In [ ]:
# Calculate ke and pathlength for all observations
vod_cell['ke'], vod_cell['pathlength'] = ke_fun(
    vod_cell['VOD1'],
    d=20,  # Canopy height in meters
    elevation=vod_cell["Elevation"]
)

# Set index for later anomaly calculation
vod_cell.set_index(['Epoch', 'SV'], inplace=True)

# Calculate cell averages with temporal resolution of 60 minutes
_, avg = calculate_anomaly(vod_cell, ["ke"], 60)

# Display summary statistics
print("Summary statistics of calculated parameters:")
print(avg[['VOD1_mean', 'ke_mean', 'pathlength_mean', 'Elevation_mean']].describe())

In [ ]:
# Create visualization grid
fig, axes = plt.subplots(2, 2, figsize=(8,8))

# VOD vs ke
avg.plot(kind="scatter", x="VOD1_mean", y="ke_mean", s=3, ax=axes[0, 0])
axes[0, 0].set_title("VOD vs Extinction Coefficient")
axes[0, 0].grid(True, alpha=0.3)

# Normalize pathlength by canopy height for better interpretation
avg["path_per_d"] = avg["pathlength_mean"] / d

# ke vs normalized pathlength
avg.plot(kind="scatter", x="ke_mean", y="path_per_d", s=3, ax=axes[0, 1])
axes[0, 1].set_title("Extinction Coefficient vs Normalized Pathlength")
axes[0, 1].set_ylabel(f"Pathlength / Canopy Height")
axes[0, 1].grid(True, alpha=0.3)

# VOD vs normalized pathlength
avg.plot(kind="scatter", x="VOD1_mean", y="path_per_d", s=3, ax=axes[1, 0])
axes[1, 0].set_title("VOD vs Normalized Pathlength")
axes[1, 0].set_ylabel(f"Pathlength / Canopy Height")
axes[1, 0].grid(True, alpha=0.3)

# Elevation vs VOD
avg.plot(kind="scatter", x="Elevation_mean", y="VOD1_mean", s=3, ax=axes[1, 1])
axes[1, 1].set_title("Elevation vs VOD")
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Visualize Hemispheric Distributions

In [ ]:
# Create a figure with 3 subplots for hemispheric distributions
# Plot the hemispheric distribution of VOD
plot_hemi(avg, hemi.patches(), "VOD1_mean", ax=axes[0],
          title="VOD Distribution", clim="auto")

# Plot the hemispheric distribution of extinction coefficient
plot_hemi(avg, hemi.patches(), "ke_mean", ax=axes[1],
          title="Extinction Coefficient", clim="auto")

# Plot the hemispheric distribution of path length
plot_hemi(avg, hemi.patches(), "pathlength_mean", ax=axes[2],
          title="Path Length (m)", clim="auto")
